# 🛒 Customer Intelligence Report — Online Retail II
**Author:** Asim Jamal  
**Dataset:** Online Retail II (UCI / Kaggle)  
**Tool:** Python (Jupyter Notebook)  
**Goal:** Perform end-to-end EDA and surface actionable marketing insights through RFM segmentation, cohort analysis, and revenue storytelling.

---

## Project Roadmap
| Phase | Description |
|---|---|
| 1️⃣ Setup & Load | Environment setup, data loading, first look |
| 2️⃣ Data Cleaning | Handle nulls, cancellations, bad records |
| 3️⃣ Revenue Analysis | Sales trends, top products, top markets |
| 4️⃣ RFM Segmentation | Customer scoring and segment labeling |
| 5️⃣ Cohort Analysis | Retention heatmap by acquisition month |
| 6️⃣ Insights Summary | Business narrative and recommendations |

---

## Phase 1 — Environment Setup & Data Loading

Before anything else, we:
1. Import all libraries we'll need throughout the project
2. Configure display settings
3. Load the dataset and take an honest first look at its shape, types, and quality

> 📌 **Dataset source:** Download `online_retail_II.csv` from [Kaggle](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci)

In [1]:
# ── 1.1  Library Imports ──────────────────────────────────────────────────────

# Core
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

# SQL-style queries inside notebooks
import sqlite3

# Suppress harmless warnings
import warnings
warnings.filterwarnings('ignore')

print('✅ All libraries imported successfully')

✅ All libraries imported successfully


In [2]:
# ── 1.2  Display & Style Configuration ───────────────────────────────────────

# Pandas display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Plot style
sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.titleweight'] = 'bold'

print('✅ Display settings configured')

✅ Display settings configured


In [3]:
# ── 1.3  Load the Dataset ─────────────────────────────────────────────────────
#
# The file is a CSV: online_retail_II.csv
# Place it in the same folder as this notebook before running.
#
# encoding='latin-1' handles special characters (e.g. accented product names)
# that will cause a UnicodeDecodeError with the default utf-8 encoding.

FILE_PATH = 'online_retail_II.csv'

print('Loading dataset ...')
df = pd.read_csv(FILE_PATH, encoding='latin-1')

print(f'\n✅ Data loaded — {df.shape[0]:,} rows × {df.shape[1]} columns')

Loading dataset ...

✅ Data loaded — 1,067,371 rows × 8 columns


In [4]:
# ── 1.4  First Look — Schema & Sample Rows ────────────────────────────────────
print('=== COLUMN NAMES & DTYPES ===')
print(df.dtypes)
print()
df.head(10)

=== COLUMN NAMES & DTYPES ===
Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,"13,085.00",United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,"13,085.00",United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,"13,085.00",United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,"13,085.00",United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,"13,085.00",United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,"13,085.00",United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,"13,085.00",United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,"13,085.00",United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,"13,085.00",United Kingdom


In [5]:
# ── 1.5  Shape & Basic Stats ──────────────────────────────────────────────────
print(f'Rows    : {df.shape[0]:,}')
print(f'Columns : {df.shape[1]}')
print(f'\nDate range: {df["InvoiceDate"].min()} → {df["InvoiceDate"].max()}')
print(f'Unique Customers : {df["Customer ID"].nunique():,}')
print(f'Unique Products  : {df["StockCode"].nunique():,}')
print(f'Unique Countries : {df["Country"].nunique():,}')

Rows    : 1,067,371
Columns : 8

Date range: 2009-12-01 07:45:00 → 2011-12-09 12:50:00
Unique Customers : 5,942
Unique Products  : 5,305
Unique Countries : 43


In [6]:
# ── 1.6  Missing Values Audit ─────────────────────────────────────────────────

missing = pd.DataFrame({
    'Missing Count' : df.isnull().sum(),
    'Missing %'     : (df.isnull().sum() / len(df) * 100).round(2)
})

print('=== MISSING VALUES ===')
print(missing[missing['Missing Count'] > 0])

=== MISSING VALUES ===
             Missing Count  Missing %
Description           4382       0.41
Customer ID         243007      22.77


In [7]:
# ── 1.7  Duplicate Check ──────────────────────────────────────────────────────

dupes = df.duplicated().sum()
print(f'Duplicate rows: {dupes:,}')

# Quick look at a few duplicates if they exist
if dupes > 0:
    print('\nSample duplicate rows:')
    display(df[df.duplicated(keep=False)].head(6))

Duplicate rows: 34,335

Sample duplicate rows:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
362,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,"16,329.00",United Kingdom
363,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,"16,329.00",United Kingdom
365,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,"16,329.00",United Kingdom
367,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,"16,329.00",United Kingdom
368,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,"16,329.00",United Kingdom
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,"16,329.00",United Kingdom


In [8]:
# ── 1.8  Numerical Columns — Sanity Check ─────────────────────────────────────
#
# We want to spot obviously bad values before we clean:
# negative quantities (returns/cancellations) and zero/negative prices.

print('=== QUANTITY distribution ===')
print(df['Quantity'].describe())
print(f'\nRows with Quantity <= 0 : {(df["Quantity"] <= 0).sum():,}')

print('\n=== PRICE distribution ===')
print(df['Price'].describe())
print(f'\nRows with Price <= 0    : {(df["Price"] <= 0).sum():,}')

=== QUANTITY distribution ===
count   1,067,371.00
mean            9.94
std           172.71
min       -80,995.00
25%             1.00
50%             3.00
75%            10.00
max        80,995.00
Name: Quantity, dtype: float64

Rows with Quantity <= 0 : 22,950

=== PRICE distribution ===
count   1,067,371.00
mean            4.65
std           123.55
min       -53,594.36
25%             1.25
50%             2.10
75%             4.15
max        38,970.00
Name: Price, dtype: float64

Rows with Price <= 0    : 6,207


## ✅ Phase 1 Complete — What We Know So Far

After loading the data, we now have a clear picture of what we're working with:

- **~1M rows** of transactional data spanning two years across multiple countries
- **Key columns:** `Invoice`, `StockCode`, `Description`, `Quantity`, `InvoiceDate`, `Price`, `Customer ID`, `Country`
- **Known issues to tackle in Phase 2:**
  - `Customer ID` has significant missing values — these rows can't be used for customer-level analysis
  - `Description` has some nulls — minor, but worth noting
  - Negative `Quantity` values indicate **returns / cancellations** (invoices starting with 'C')
  - Some `Price` values are zero or negative — need investigation
  - Duplicate rows exist and need to be removed

> 🔜 **Next up — Phase 2: Data Cleaning.** We'll resolve every issue flagged above before a single chart is drawn.

## Phase 2 — Data Cleaning

Based on our Phase 1 audit, we have 6 concrete steps to resolve in a deliberate order:

| Step | Issue | Rows Affected | Action |
|---|---|---|---|
| 2.1 | Duplicate rows | TBD | Drop exact duplicates |
| 2.2 | Cancellations / Returns | ~22,950 | Isolate and remove (Invoice starts with 'C') |
| 2.3 | Missing Customer ID | 243,007 (22.77%) | Drop — required for all customer analysis |
| 2.4 | Bad Price & Quantity | ~6,207 | Remove zero/negative values |
| 2.5 | Data types | All rows | Parse InvoiceDate, fix Customer ID type |
| 2.6 | Feature engineering | All rows | Add TotalRevenue, Year, Month, YearMonth |

> 📌 Raw `df` is always preserved. All cleaning happens on `df_clean` so every decision is traceable.

In [9]:
# ── 2.0  Preserve the Raw DataFrame ──────────────────────────────────────────

df_clean = df.copy()
print(f'Starting shape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns')

Starting shape: 1,067,371 rows × 8 columns


In [10]:
# ── 2.1  Remove Duplicate Rows ────────────────────────────────────────────────

before = len(df_clean)
df_clean = df_clean.drop_duplicates()
dropped = before - len(df_clean)

print(f'Duplicates removed : {dropped:,}')
print(f'Rows remaining     : {len(df_clean):,}')

Duplicates removed : 34,335
Rows remaining     : 1,033,036


In [11]:
# ── 2.2  Handle Cancellations & Returns ───────────────────────────────────────
#
# Cancelled invoices start with 'C' (e.g. C489449).
# Saved separately in df_cancelled for optional future analysis
# (e.g. return rate by product or country).

df_cancelled = df_clean[df_clean['Invoice'].astype(str).str.startswith('C')].copy()

before = len(df_clean)
df_clean = df_clean[~df_clean['Invoice'].astype(str).str.startswith('C')]
dropped = before - len(df_clean)

print(f'Cancellation rows removed          : {dropped:,}')
print(f'Cancellations saved in df_cancelled: {len(df_cancelled):,} rows')
print(f'Rows remaining                     : {len(df_clean):,}')

Cancellation rows removed          : 19,104
Cancellations saved in df_cancelled: 19,104 rows
Rows remaining                     : 1,013,932


In [12]:
# ── 2.3  Remove Rows with Missing Customer ID ─────────────────────────────────
#
# Guest/unregistered transactions have no Customer ID.
# Dropping them is a documented trade-off — not a data loss mistake.
# RFM and cohort analysis are impossible without a customer identifier.

before = len(df_clean)
df_clean = df_clean.dropna(subset=['Customer ID'])
dropped = before - len(df_clean)

print(f'Rows dropped (missing Customer ID) : {dropped:,}')
print(f'Rows remaining                     : {len(df_clean):,}')

Rows dropped (missing Customer ID) : 234,437
Rows remaining                     : 779,495


In [13]:
# ── 2.4  Remove Bad Price & Quantity Values ───────────────────────────────────
#
# After removing cancellations, any remaining Quantity <= 0
# or Price <= 0 are data entry errors or test/adjustment records.

before = len(df_clean)

df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean = df_clean[df_clean['Price'] > 0]

dropped = before - len(df_clean)

print(f'Rows dropped (bad Quantity or Price) : {dropped:,}')
print(f'Rows remaining                       : {len(df_clean):,}')

Rows dropped (bad Quantity or Price) : 70
Rows remaining                       : 779,425


In [14]:
# ── 2.5  Fix Data Types ───────────────────────────────────────────────────────
#
# InvoiceDate → datetime  (enables all time-series operations)
# Customer ID → string    (it's an identifier, never to be averaged or summed)

df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['Customer ID'] = df_clean['Customer ID'].astype(int).astype(str)

print('Updated dtypes:')
print(df_clean[['InvoiceDate', 'Customer ID']].dtypes)

Updated dtypes:
InvoiceDate    datetime64[ns]
Customer ID            object
dtype: object


In [15]:
# ── 2.6  Feature Engineering ──────────────────────────────────────────────────
#
# Build derived columns once here — used across all remaining phases.

df_clean['TotalRevenue'] = df_clean['Quantity'] * df_clean['Price']
df_clean['Year']         = df_clean['InvoiceDate'].dt.year
df_clean['Month']        = df_clean['InvoiceDate'].dt.month
df_clean['YearMonth']    = df_clean['InvoiceDate'].dt.to_period('M')

print('New columns added: TotalRevenue, Year, Month, YearMonth')
print()
df_clean[['InvoiceDate', 'Quantity', 'Price', 'TotalRevenue', 'Year', 'Month', 'YearMonth']].head(5)

New columns added: TotalRevenue, Year, Month, YearMonth



,InvoiceDate,Quantity,Price,TotalRevenue,Year,Month,YearMonth
0,2009-12-01 07:45:00,12,6.95,83.40,2009,12,2009-12
1,2009-12-01 07:45:00,12,6.75,81.00,2009,12,2009-12
2,2009-12-01 07:45:00,12,6.75,81.00,2009,12,2009-12
3,2009-12-01 07:45:00,48,2.10,100.80,2009,12,2009-12
4,2009-12-01 07:45:00,24,1.25,30.00,2009,12,2009-12
